To use pytorch geometric temporal, make sure you have torch 1.9.0 installed (uninstall 1.10.0 before).

In [2]:
import torch
print(torch.__version__)

1.9.0+cpu


In [3]:
import torch
import numpy as np
import pandas as pd

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch-geometric
!pip install torch-geometric-temporal

In [4]:
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

def transform_and_split(data):
    # Normalize node features and transform data type
    data.x = normalize(data.x, axis=1, norm='max')
    data.x = torch.from_numpy(data.x).to(torch.float64)
    data.y = data.y.apply_(lambda x:  1 if (x > 0) else 0) # Change y into {0, 1} for binary classification
    data.y = data.y.to(torch.float64)    
    data.edge_attr = data.edge_attr.to(torch.double)


    # Split into train/test set
#    split = nodeSplit(split="train_rest", num_splits = 1, num_val = 0.0, num_test= 0.2)
#    masked_data = split(data)

#    print("Training samples:", torch.sum(masked_data.train_mask).item())
#    print("Validation samples:", torch.sum(masked_data.val_mask ).item())
#    print("Test samples:", torch.sum(masked_data.test_mask ).item())
    print_basic_info(data)
    return data

In [5]:
def print_basic_info(data):
    print()
    print(data)
    print('===========================================================================================================')

    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

### Get and split data

In [6]:
path = "../data/processed/twitter/2018_q1.pt" # Customize...
dataset = torch.load(path)
data = dataset[0]
transformed_data = transform_and_split(data)


Data(x=[29, 61], edge_index=[2, 400], edge_attr=[400], y=[29])
Number of nodes: 29
Number of edges: 400
Average node degree: 13.79
Has isolated nodes: False
Has self-loops: True
Is undirected: True


In [15]:
df = pd.read_csv('../data/raw/news/news_data_weekly.csv')

In [16]:
df = df.dropna()

In [17]:
df = df.sort_values('Date')

In [18]:
df

,Date,url,texts
2746,2016-10-08,https://www.nytimes.com/2016/10/04/opinion/the...,Donald Trump is a thug. He’s a thug who talks...
33,2016-10-08,https://www.nytimes.com/2016/10/08/us/politics...,In lucrative paid speeches that Hillary Clint...
32,2016-10-08,https://www.nytimes.com/2016/10/07/technology/...,"SAN FRANCISCO — Marc Benioff, the founder and..."
31,2016-10-08,https://www.nytimes.com/2016/10/05/business/pr...,Prepaid debit cards are a financial lifeline ...
30,2016-10-08,https://www.nytimes.com/2016/10/04/world/ameri...,RIO DE JANEIRO — It was not a banner day for ...
...,...,...,...
1518,2021-10-02,https://www.nytimes.com/2021/09/30/sports/socc...,Looking to expand its global footprint beyond...
1517,2021-10-02,https://www.nytimes.com/2021/10/01/business/cr...,Despite the popularity of mobile apps promisi...
1516,2021-10-02,https://www.nytimes.com/2021/10/02/your-money/...,Introducing your child to the real-world use ...
2736,2021-10-02,https://www.nytimes.com/2021/09/26/fashion/wat...,Like their counterparts in industries such as...


In [19]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [20]:
df_list = [part for _, part in df.groupby(pd.Grouper(key='Date', freq='W-MON'))]

In [21]:
df_list[0]

,Date,url,texts
2746,2016-10-08,https://www.nytimes.com/2016/10/04/opinion/the...,Donald Trump is a thug. He’s a thug who talks...
33,2016-10-08,https://www.nytimes.com/2016/10/08/us/politics...,In lucrative paid speeches that Hillary Clint...
32,2016-10-08,https://www.nytimes.com/2016/10/07/technology/...,"SAN FRANCISCO — Marc Benioff, the founder and..."
31,2016-10-08,https://www.nytimes.com/2016/10/05/business/pr...,Prepaid debit cards are a financial lifeline ...
30,2016-10-08,https://www.nytimes.com/2016/10/04/world/ameri...,RIO DE JANEIRO — It was not a banner day for ...
36,2016-10-08,https://www.nytimes.com/2016/10/07/business/de...,WASHINGTON — Nearly five years after Jon S. C...
37,2016-10-08,https://www.nytimes.com/2016/10/08/business/in...,LONDON — As Europe has grappled with the trau...
38,2016-10-08,https://www.nytimes.com/2016/10/08/world/europ...,LONDON — For those blithely inclined toward t...
34,2016-10-08,https://www.nytimes.com/2016/10/04/business/de...,The Janus Capital Group and the Henderson Gro...
2747,2016-10-08,https://www.nytimes.com/2016/10/09/world/middl...,TEHRAN — Rushing for a plane to Tehran becaus...


In [25]:
import yaml

with open('../configs/dow_jones.yaml') as f:
    
    data = yaml.load(f, Loader=yaml.FullLoader)
    print(data)

{'companies': [{'wba': {'alias': ['$wba', 'wba', 'walgreen boots alliance inc', 'walgreen boots alliance', 'walgreenbootsalliance']}}, {'v': {'alias': ['$v', 'v', 'visa inc class a', 'visa']}}, {'crm': {'alias': ['$crm', 'crm', 'salesforce.com inc', 'salesforce']}}, {'cvx': {'alias': ['$cvx', 'cvx', 'chevron corp', 'chevron']}}, {'pg': {'alias': ['$pg', 'pg', 'procter & gamble', 'procter&gamble']}}, {'vz': {'alias': ['$vz', 'vz', 'verizon communications inc', 'verizon']}}, {'wmt': {'alias': ['$wmt', 'wmt', 'walmart stores inc', 'walmart stores', 'walmart']}}, {'unh': {'alias': ['$unh', 'unh', 'unitedhealth group inc', 'unitedhealth group', 'unitedhealthgroup']}}, {'trv': {'alias': ['$trv', 'trv', 'travelers companies inc', 'travelers companies', 'travelers', 'travelerscompanies']}}, {'mcd': {'alias': ['$mcd', 'mcd', 'mcdonalds corp', 'mcdonalds']}}, {'mmm': {'alias': ['$mmm', 'mmm', '3m', '3m']}}, {'nke': {'alias': ['$nke', 'nke', 'nike inc class b', 'nike']}}, {'mrk': {'alias': ['$mrk

In [27]:
def get_matrix(df):
    companies = [list(com.keys())[0] for com in data['companies']]
    alias = list(map(lambda x: list(x.items())[0][1]["alias"], data['companies']))
    res = pd.DataFrame(0, index=companies, columns=companies)
    for company1, search_items1 in zip(companies, alias):
        for company2, search_items2 in zip(companies, alias):
            if company1 != company2:
                search_items = search_items1 + search_items2
            else:
                search_items = search_items1
            pat = "|".join(search_items)
            res[company1][company2] += df.texts.str.contains(
                pat
            ).sum()
    return res.values

In [28]:
mat_list = [get_matrix(df) for df in df_list]

In [29]:
len(mat_list)

261

In [30]:
week = 0
for i in range(261):
    np.save('../data/raw/news/week_'+str(week)+'.npy', mat_list[i])
    week += 1
        

In [31]:
import os
stock_df = pd.read_csv(
            os.path.join('../data/raw',"stock","raw.csv"),
            usecols=["ticker_symbol", "Date", "Close"],
            parse_dates=["Date"],
        )

In [32]:
stock_df

,Date,Close,ticker_symbol
0,2016-10-03,28.129999,aapl
1,2016-10-04,28.250000,aapl
2,2016-10-05,28.262501,aapl
3,2016-10-06,28.472500,aapl
4,2016-10-07,28.514999,aapl
...,...,...,...
36506,2021-09-27,142.250000,wmt
36507,2021-09-28,140.500000,wmt
36508,2021-09-29,140.440002,wmt
36509,2021-09-30,139.380005,wmt


In [56]:
start = datetime.strptime('2016-10-02', "%Y-%m-%d")

In [57]:
from datetime import timedelta

In [58]:
date_list = [start]
cur = start
for i in range(261):
    cur = cur + timedelta(days=7)
    date_list.append(cur)

In [59]:
date_list

[datetime.datetime(2016, 10, 2, 0, 0),
 datetime.datetime(2016, 10, 9, 0, 0),
 datetime.datetime(2016, 10, 16, 0, 0),
 datetime.datetime(2016, 10, 23, 0, 0),
 datetime.datetime(2016, 10, 30, 0, 0),
 datetime.datetime(2016, 11, 6, 0, 0),
 datetime.datetime(2016, 11, 13, 0, 0),
 datetime.datetime(2016, 11, 20, 0, 0),
 datetime.datetime(2016, 11, 27, 0, 0),
 datetime.datetime(2016, 12, 4, 0, 0),
 datetime.datetime(2016, 12, 11, 0, 0),
 datetime.datetime(2016, 12, 18, 0, 0),
 datetime.datetime(2016, 12, 25, 0, 0),
 datetime.datetime(2017, 1, 1, 0, 0),
 datetime.datetime(2017, 1, 8, 0, 0),
 datetime.datetime(2017, 1, 15, 0, 0),
 datetime.datetime(2017, 1, 22, 0, 0),
 datetime.datetime(2017, 1, 29, 0, 0),
 datetime.datetime(2017, 2, 5, 0, 0),
 datetime.datetime(2017, 2, 12, 0, 0),
 datetime.datetime(2017, 2, 19, 0, 0),
 datetime.datetime(2017, 2, 26, 0, 0),
 datetime.datetime(2017, 3, 5, 0, 0),
 datetime.datetime(2017, 3, 12, 0, 0),
 datetime.datetime(2017, 3, 19, 0, 0),
 datetime.datetime(2

In [138]:
comp_emb.shape

torch.Size([29, 768])

In [136]:
comp_emb = []
for fp in sorted(os.listdir("../data/raw/sec/")):
    full_path = os.path.join("../data/raw", "sec", fp)
    if fp.split(".")[-1]=='npy':
        comp_emb.append(torch.from_numpy(np.load(full_path)))
comp_emb = torch.stack(comp_emb)

In [63]:
X_y = []
for i in range(260):
    start_date = date_list[i]
    end_date = start_date+timedelta(days=6)
    next_start_date = start_date+timedelta(days=7)
    next_end_date = start_date+timedelta(days=13)
    ######################################################## 
    # prepare X (change this if you want to add SEC emb, etc.)
    ########################################################
    curr = stock_df[(stock_df.Date>=start_date) & (stock_df.Date<=end_date)]
    X = curr.pivot_table(
            index="Date", columns="ticker_symbol", values="Close"
        ).values.T
    X_tensor = torch.tensor(X)
    
    ########################################################
    # prepare y (change this if you want to change labels)
    ########################################################

    
    nxt = stock_df[(stock_df.Date>=next_start_date) & (stock_df.Date<=next_end_date)]
    y = nxt.pivot_table(
            index="Date", columns="ticker_symbol", values="Close"
        ).values.T
    y = (y.mean(1) - X.mean(1)) / X.mean(1)
    y_tensor = torch.tensor(y)
    X_y.append((X_tensor,y_tensor))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259


In [305]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

In [98]:
dataset.edge_weight

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [99]:
path = "../data/processed/twitter/"

In [100]:
quarter = ['2016_q4']
for i in range(2017, 2022):
    for j in range(1, 5):
        if i == 2021 and j == 4: break
        quarter.append(str(i)+'_q'+str(j))

In [101]:
quarter

['2016_q4',
 '2017_q1',
 '2017_q2',
 '2017_q3',
 '2017_q4',
 '2018_q1',
 '2018_q2',
 '2018_q3',
 '2018_q4',
 '2019_q1',
 '2019_q2',
 '2019_q3',
 '2019_q4',
 '2020_q1',
 '2020_q2',
 '2020_q3',
 '2020_q4',
 '2021_q1',
 '2021_q2',
 '2021_q3']

In [102]:
paths = []
for i in quarter:
    paths.append(path+i+'.pt')

In [103]:
paths

['../data/processed/twitter/2016_q4.pt',
 '../data/processed/twitter/2017_q1.pt',
 '../data/processed/twitter/2017_q2.pt',
 '../data/processed/twitter/2017_q3.pt',
 '../data/processed/twitter/2017_q4.pt',
 '../data/processed/twitter/2018_q1.pt',
 '../data/processed/twitter/2018_q2.pt',
 '../data/processed/twitter/2018_q3.pt',
 '../data/processed/twitter/2018_q4.pt',
 '../data/processed/twitter/2019_q1.pt',
 '../data/processed/twitter/2019_q2.pt',
 '../data/processed/twitter/2019_q3.pt',
 '../data/processed/twitter/2019_q4.pt',
 '../data/processed/twitter/2020_q1.pt',
 '../data/processed/twitter/2020_q2.pt',
 '../data/processed/twitter/2020_q3.pt',
 '../data/processed/twitter/2020_q4.pt',
 '../data/processed/twitter/2021_q1.pt',
 '../data/processed/twitter/2021_q2.pt',
 '../data/processed/twitter/2021_q3.pt']

In [104]:
data_list = []

In [105]:
for path in paths:
    dataset = torch.load(path)
    data = dataset[0]
    data_list.append(transform_and_split(data))


Data(x=[29, 63], edge_index=[2, 760], edge_attr=[760], y=[29])
Number of nodes: 29
Number of edges: 760
Average node degree: 26.21
Has isolated nodes: False
Has self-loops: True
Is undirected: True

Data(x=[29, 62], edge_index=[2, 312], edge_attr=[312], y=[29])
Number of nodes: 29
Number of edges: 312
Average node degree: 10.76
Has isolated nodes: False
Has self-loops: True
Is undirected: True

Data(x=[29, 63], edge_index=[2, 400], edge_attr=[400], y=[29])
Number of nodes: 29
Number of edges: 400
Average node degree: 13.79
Has isolated nodes: False
Has self-loops: True
Is undirected: True

Data(x=[29, 63], edge_index=[2, 552], edge_attr=[552], y=[29])
Number of nodes: 29
Number of edges: 552
Average node degree: 19.03
Has isolated nodes: False
Has self-loops: True
Is undirected: True

Data(x=[29, 63], edge_index=[2, 805], edge_attr=[805], y=[29])
Number of nodes: 29
Number of edges: 805
Average node degree: 27.76
Has isolated nodes: False
Has self-loops: True
Is undirected: True

Data

In [106]:
len(data_list)

20

In [107]:
data_list[1].x.shape

torch.Size([29, 62])

In [108]:
"""
edge_indices = [i.edge_index.double() for i in data_list]
edge_weights = [i.edge_attr.double() for i in data_list]
features = [i.x.double() for i in data_list]
targets = [i.y.double() for i in data_list]
"""
"""
edge_indices = [i.edge_index.cpu().detach().numpy() for i in data_list]
edge_weights = [i.edge_attr.cpu().detach().numpy() for i in data_list]
features = [i.x.cpu().detach().numpy() for i in data_list]
targets = [i.y.cpu().detach().numpy() for i in data_list]
"""
edge_indices = [i.edge_index.numpy() for i in data_list]
edge_weights = [i.edge_attr.numpy() for i in data_list]
features = [i.x.numpy() for i in data_list]
targets = [i.y.numpy() for i in data_list]

In [109]:
edge_indices

[array([[ 0,  0,  0, ..., 28, 28, 28],
        [ 2,  3,  4, ..., 26, 27, 28]], dtype=int64),
 array([[ 0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,
          2,  2,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,
          6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  9,  9,  9,
          9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
          9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12,
         12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15,
         15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
         17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20,
         20, 20, 20, 21, 21, 21

In [84]:
from torch_geometric.utils import dense_to_sparse
edge_idx = []
edge_att = []
for i in range(260):
    edge_index, edge_attr = dense_to_sparse(torch.from_numpy(mat_list[i]))
    edge_idx.append(edge_index)
    edge_att.append(edge_attr)

In [85]:
edge_indices = [i.numpy() for i in edge_idx]
edge_weights = [i.numpy() for i in edge_att]

In [96]:
features = []
targets = []
for i in range(260):
    features.append(normalize(X_y[i][0].numpy(), axis=1, norm='max'))
    #features.append(X_y[i][0].numpy())
    targets.append([1 if a > 0 else 0 for a in X_y[i][1].numpy()])
targets = np.asarray(targets)

In [97]:
targets[3]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0])

In [98]:
mat_list[2].shape

(29, 29)

In [99]:
padded_features[1].shape

(29, 5)

In [100]:
padded_features = []
for i in features:
    padded_features.append(np.pad(i, [(0, 0), (0, 5-i.shape[1])], 'mean'))

In [150]:
padded_features = np.asarray(padded_features)

In [162]:
padded_features.shape

(260, 29, 773)

In [154]:
comp_emb = np.asarray([comp_emb.numpy() for i in range(260)])

In [159]:
comp_emb.shape

(260, 29, 768)

In [161]:
padded_features = np.concatenate((padded_features, comp_emb), axis = 2)

In [163]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal

In [164]:
len(padded_features)

260

In [165]:
temporal_signal = DynamicGraphTemporalSignal(edge_indices = edge_indices , edge_weights = edge_weights, features = padded_features, targets = targets)

In [166]:
temporal_signal

In [167]:
from torch_geometric_temporal.signal import temporal_signal_split

train_dataset, test_dataset = temporal_signal_split(temporal_signal, train_ratio=0.8)

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN
from torch_geometric_temporal import GConvGRU
from torch_geometric_temporal import EvolveGCNO
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.evol = EvolveGCNO(node_features)
        self.recurrent = DCRNN(node_features, 16, 1)
        self.conv = GConvGRU(node_features, 64, 3)
        #self.linear = torch.nn.Linear(16, 1)
        self.linear = torch.nn.Linear(64, 2)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge_index, edge_weight):
#        h = self.recurrent(x, edge_index, edge_weight)
#        h = self.dropout(h)
        h = self.conv(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        h = torch.sigmoid(h)
        return h

In [2]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 773)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    loss = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        loss += torch.nn.CrossEntropyLoss()(y_hat, snapshot.y.long())
#        loss += torch.mean((y_hat-snapshot.y)**2)
#        loss = loss / (time+1)
        
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/200 [00:00<?, ?it/s]


NameError: name 'train_dataset' is not defined

In [174]:
y_hat_l = []
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    #cost = cost + torch.mean((y_hat-snapshot.y)**2)
    y_hat_l.append(y_hat)
#cost = cost / (time+1)
#cost = cost.item()
#print("MSE: {:.4f}".format(cost))


In [175]:
y_hat_l

[tensor([[0.4215, 0.6706],
         [0.4884, 0.7967],
         [0.3832, 0.7016],
         [0.4128, 0.6792],
         [0.4484, 0.6509],
         [0.4014, 0.6830],
         [0.4192, 0.6690],
         [0.4508, 0.6499],
         [0.5059, 0.6028],
         [0.4601, 0.6401],
         [0.4281, 0.6657],
         [0.3131, 0.7543],
         [0.4349, 0.6609],
         [0.4594, 0.6436],
         [0.3904, 0.6933],
         [0.4468, 0.7603],
         [0.4244, 0.6690],
         [0.4884, 0.7967],
         [0.4508, 0.6468],
         [0.4884, 0.7967],
         [0.4012, 0.6836],
         [0.5265, 0.5900],
         [0.4884, 0.7967],
         [0.4919, 0.7304],
         [0.3989, 0.6871],
         [0.4375, 0.6598],
         [0.4884, 0.7967],
         [0.4311, 0.6632],
         [0.3883, 0.6926]], grad_fn=<SigmoidBackward>),
 tensor([[0.3099, 0.7534],
         [0.4536, 0.8277],
         [0.2411, 0.8074],
         [0.2988, 0.7656],
         [0.3509, 0.7250],
         [0.2626, 0.7824],
         [0.2875, 0.7635],

In [176]:
y_hat_l = [list(np.squeeze(i.detach().numpy())) for i in y_hat_l]
y_hat_l = [z for y in y_hat_l for z in y]

In [177]:
y_hat_l = [y[1] for y in y_hat_l]

In [178]:
y_hat_l

[0.67061865,
 0.7966896,
 0.70161974,
 0.67922944,
 0.6509174,
 0.68296516,
 0.66900086,
 0.64990574,
 0.6028081,
 0.6401323,
 0.66565436,
 0.75427705,
 0.66087997,
 0.64360666,
 0.69332373,
 0.7603342,
 0.6689626,
 0.7966896,
 0.6467989,
 0.7966896,
 0.68361264,
 0.5899542,
 0.7966896,
 0.7304265,
 0.6871339,
 0.65983117,
 0.7966896,
 0.6631817,
 0.69259286,
 0.7534302,
 0.82766867,
 0.8074023,
 0.7656217,
 0.7249651,
 0.78235877,
 0.7635128,
 0.7355015,
 0.62989587,
 0.7031404,
 0.742613,
 0.82766867,
 0.75561064,
 0.70897025,
 0.7987101,
 0.62970644,
 0.75784475,
 0.82766867,
 0.72149986,
 0.8448559,
 0.7798597,
 0.58986133,
 0.75889605,
 0.82766867,
 0.78079814,
 0.7315216,
 0.82766867,
 0.74217516,
 0.7833304,
 0.66491127,
 0.7529848,
 0.6768209,
 0.6695302,
 0.68941957,
 0.6685791,
 0.667408,
 0.6605773,
 0.6420391,
 0.6525958,
 0.6633989,
 0.66022336,
 0.6628683,
 0.65599704,
 0.675304,
 0.66022336,
 0.6651538,
 0.7529848,
 0.6590826,
 0.6791147,
 0.6713051,
 0.7077246,
 0.75069

In [179]:
import numpy as np
from sklearn import metrics
y = true_label
pred = np.array(y_hat_l)
fpr, tpr, thresholds = metrics.roc_curve(y, pred)
metrics.auc(fpr, tpr)

0.5106382408679204

In [206]:
y_hat_list = [1 if x > 0.70 else 0 for x in y_hat_l]

In [207]:
true_label = []
for time, snapshot in enumerate(test_dataset):
    true_label.append(list(snapshot.y.detach().numpy()))

In [208]:
true_label = [int(z) for y in true_label for z in y]

In [209]:
from sklearn.metrics import classification_report
y_true = true_label
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_hat_list, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.45      0.51      0.48       661
     class 1       0.57      0.51      0.54       847

    accuracy                           0.51      1508
   macro avg       0.51      0.51      0.51      1508
weighted avg       0.52      0.51      0.51      1508

